In [ ]:
import os
os.chdir('..')

In [ ]:
import torch

In [ ]:
import utils
from dataloader.data import Feature, MIMICDataset, Normalizer
from dataloader.extractors import get_vocab
from dataloader.utils import BinnedEvent

In [ ]:
DEVICE = 'cuda'

In [ ]:
chart = Feature('CHARTEVENTS', 'LABEL', 'VALUE', event_class=BinnedEvent, special_tok='🅒🅔', **params)
lab = Feature('LABEVENTS', 'LABEL', 'VALUE', event_class=BinnedEvent, special_tok='🅛🅔', **params)
output = Feature('OUTPUTEVENTS', 'LABEL', 'VALUE', event_class=BinnedEvent, special_tok='🅞🅔', **params)
dem = Normalizer('dem', normalized_indices=[0, 1, 2])

tables = [chart, lab, output, dem]

train_set = MIMICDataset(datalist_file='train_listfile.csv', mode='TRAIN',
                         tables=tables,
                         limit=None,
                         numericalize=False,
                         )

train_set.numericalize = True

joint_vocab = get_vocab(**params)

chart.load_params(**params)
chart.vocab = joint_vocab
lab.load_params(**params)
lab.vocab = joint_vocab
output.load_params(**params)
output.vocab = joint_vocab
dem.load_params(**params)

In [ ]:
params

In [ ]:
params = utils.load_params('pbwo3owi')
model = utils.load_model(params, joint_vocab, tables, DEVICE)

In [ ]:
with torch.no_grad():
    embeddings = model.timestep_encoder.event_encoder.encoder.weight
    embeddings = torch.cat([embeddings, torch.zeros(embeddings.shape[0], 2)], 1)
    embeddings = model.timestep_encoder.model.linear(embeddings)

In [ ]:
sorted_emb, sorted_emb_ind = embeddings.sort(0, descending=True)

In [ ]:
top_k = 10

for i in range(sorted_emb.shape[1]):
    print(f"### DIMENSION {i} ###")
    indices = sorted_emb_ind[:top_k,i].int().numpy()
    for ind in indices:
        print(joint_vocab.itos[ind], sorted_emb[ind, i].item())